# Assigment 4

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

In this path **..\_data\sbs\B_RawData\bancos** you will find scraped data from [this link](https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=B). We get all the information of the last available day of every each month.

1. Import all the data and generate a column named as `date_info` that should have the day to information corresponds.
2. Append all this datasets and generate a unique dataframe. This newdataset should have information at `rate interest` and `date` level. The columns should be the name of the banks. Be careful since not all the excel files have the same format. **It is totally prohibited to manipulate manually the excel files. This kind of action means NO GRADE on this project.**
3. What are the top 5 banks each year with the highest interest rate at `Préstamos hipotecarios para vivienda`, `Consumo -
Tarjetas de Crédito`. Present a dataframe with these variables: `year`, `rate_concept`, `banks`, `rate_value`.
4. We want to save this dataset in the folder **_output/sbs/group_#**, but we want to save a file per bank. We want to have the information disaggregated at the bank level. Please, save your files with the name of the bank. Avoid blank spaces and use only lowercase letters. Generate the folder of your group using python code. **Hint: os library**

In [1]:
import pandas as pd
import datetime as date
import os
from pathlib import Path


# Answer 1:

In [2]:
# Primero se realizó una revisión de la carpeta conteniendo los datos. Se observa que hay datos desde el 2001 hasta el 2022 de las tasas de interés de los
# bancos. Además, existen distintas formas de los formatos. En particular, hay un formato traspuesto desde el 2002 hasta agosto de 2010 y luego un formato
# donde los bancos son las columnas a partir de septiembre de 2010. El caso del 2001 (solo hay una base de dicho año) tiene el formato de los últimos años.

# Como la data presentada está a nivel mensual, pero no contiene información de todos los meses de los años, se procede a hacer un bucle for para cada año, mes y
# día. Sin embargo, para evitar que salga error y deje de correr el bucle, se utiliza el comando 'try' y 'except' para que cuando en la base de datos no tenga la
# combinación requerida de año, mes y día, pase al siguiente paso del bucle.

# Finalmente, a partir del bucle mencionado se importará cada uno de los excel que contienen la información mensual de las tasas de interés de los bancos y se generará un DataFrame (df) por cada mes identificándolo con un número generado a partir de un contador. Este contador nos servirá para: identificar el df donde empieza y termina el cambio de formato de los excel.

contador = 0
for anio in range(2001, 2023):
    for mes in range(1,13):
        for dia in range(26,32):
            try:
                df = pd.read_excel(r"../../_data\sbs\B_RawData\bancos\table_clean_"+str(dia)+"_"+str(mes)+"_"+str(anio)+".xlsx")
                df["id_df"] = contador                          #Almacenamos el número de orden del df en una variable
                df["date_info"] = date.date(anio, mes, dia)     #Almacenamos la fecha del excel en una variable
                locals()[f'df{contador}'] = df                  #Generamos un nuevo df con el 'id' del contador
                contador+=1                                     #Hacemos que el contador crezca solo si se genera una base de datos
                if anio == 2010 and mes == 8 and dia == 31:     #Es la fecha en la que empieza cambia de formato
                    limite = contador
                elif anio == 2001 and mes == 3 and dia == 31:   #En el primer archivo no hay problema de formato
                    inicio = contador
                else:
                    continue
            except:
                continue

print(f'Se exportaron satisfactoriamente ' + str(contador) + ' archivos excel')
print(f'El formato es distinto entre el df' + str(inicio) + ' y el df' + str(limite))

Se exportaron satisfactoriamente 241 archivos excel
El formato es distinto entre el df1 y el df97


# Answer 2:

In [3]:
# Antes de poder juntar todas las bases debemos trabajar los df para el periodo donde se tiene un formato distinto. Para ello, se generará un bucle con la ayuda
# de los objetos 'inicio' y 'limite' generados en la pregunta anterior. El objetivo de este bucle es modificar el formato de las tablas entre el 2002 hasta
# agosto de 2010. Nuevamente, como no hay df para todos los números entre 'inicio' y 'límite' se utiliza el comando 'try' y 'except' para que funcione el bucle

for data in range(inicio,limite):
    alterna = locals()[f'df{data}']                #En esta línea generamos un df "comodín" denominado alterna que usaremos para cambiar el formato de los df.
    fecha = alterna.iloc[ -1 ,-1]                  #Generamos un objeto denominado fecha, que almacene la fecha de cada df
    id = alterna.iloc[-1,-2]                       #Generamos un objeto denominado id que almacena el # de contador asociado a la tabla
    alterna = alterna.rename(columns={ 'Unnamed: 0':''})
    alterna = alterna.iloc[ 1:-1 , 0:-2].T         #Transponemos el df y eliminamos la fila y id de fecha para solo quedarnos con las tasas de interés
    alterna.columns = alterna.iloc[0]              #Modificamos los index de columnas para que sean los bancos (primera fila de los df)
    alterna = alterna.iloc[1:]                     #Dropeamos la primera fila que contiene la info de los bancos
    alterna.columns = alterna.columns.str.lower()  #Modificamos los nombres de las variables para que estén en minúsculas
    alterna["Tasa Anual (%)"] = alterna.iloc[:,0]  #Generamos una variable denominada "Tasa Anual (%)" con la info de los tipos de crédito. Esto porque no nos permite modificar un NaN
    alterna = alterna.iloc[: , 1:]                 #Eliminamos la columna "NaN"
    alterna = alterna[['Tasa Anual (%)'] + [col for col in alterna.columns if col != 'Tasa Anual (%)']] #Ahora reordenamos las columnas, para que Tasa Anual sea la primera
    alterna = alterna.reset_index().iloc[: , 1:]
    alterna["date_info"] = fecha                   #Generamos la variable fecha utilizando el objeto fecha
    alterna["id_df"] = id                          #Generamos la variable id_df utilizando el objeto id
    locals()[f'df{data}'] = alterna                #Actualizamos el df con el df comodín que se generó en el bucle


In [4]:
# Como algunos meses los bancos están con mayúsculas, otros con minúsculas hemos generado un bucle para modificar las minúsculas/mayúsculas de cada variable
for data in range(0,contador):
    alterna = locals()[f'df{data}']
    alterna.columns = alterna.columns.str.lower()

In [9]:
# Creamos un df que contenga la base de datos generada y utilizamos un bucle para concatenar los dataframes
df_final = df0
for data in range(1, contador):
    df_final = pd.concat([df_final , locals()[f'df{data}']])

df_final

,tasa anual (%),bbva,comercio,crédito,pichincha,bif,scotiabank,citibank,interbank,mibanco,...,continental,financiero,azteca,deutsche,azteca *,cencosud,banco gnb,financiero *,cat,bci
0,Corporativos,2.69,5.50,2.90,5.93,4.09,3.62,-,4.22,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Descuentos,4.11,5.50,3.57,5.11,4.27,4.12,-,3.98,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Préstamos hasta 30 días,2.81,-,2.62,-,3.00,2.48,-,4.27,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Préstamos de 31 a 90 días,2.74,-,2.84,3.73,3.64,5.51,-,4.74,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Préstamos de 91 a 180 días,3.07,-,3.33,5.26,4.21,3.32,-,3.64,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,Préstamos no Revolventes para libre disponibil...,22.13,33.35,63.03,26.69,10.00,26.47,-,28.87,66.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-
42,Préstamos no Revolventes para libre disponibil...,18.22,14.66,18.99,33.05,13.39,17.84,-,21.23,37.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-
43,Créditos pignoraticios,-,42.40,-,-,-,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-
44,Hipotecarios,9.28,9.08,9.27,8.72,9.23,8.61,-,8.73,15.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-


# Answer 3:

In [6]:
# Generamos la variable año en la base de datos
cont = 0
df_final["year"] = ""
for obs in df_final['date_info']:
    df_final["year"].iloc[cont] = obs.strftime("%Y")
    cont+=1

# Se ha identificado que la forma de denominar crédito hipotecario o crédito de consumo ha variado durante el periodo de análisis, por lo que hacemos un cambio para uniformizar la denominación en toda la base de datos.
df_final.loc[df_final['tasa anual (%)'] == "HIPOTECARIO", 'tasa anual (%)'] = 'Préstamos hipotecarios para vivienda'
df_final.loc[df_final['tasa anual (%)'] == "Hipotecarios", 'tasa anual (%)'] = 'Préstamos hipotecarios para vivienda'
df_final.loc[df_final['tasa anual (%)'] == "PRÉSTAMOS HIPOTECARIOS", 'tasa anual (%)'] = 'Préstamos hipotecarios para vivienda'

df_final.loc[df_final['tasa anual (%)'] == "Consumo", 'tasa anual (%)'] = 'Consumo - Tarjeta de crédito'
df_final.loc[df_final['tasa anual (%)'] == "Tarjetas de Crédito", 'tasa anual (%)'] = 'Consumo - Tarjeta de crédito'
df_final.loc[df_final['tasa anual (%)'] == "Tarjetas de crédito", 'tasa anual (%)'] = 'Consumo - Tarjeta de crédito'
df_final.loc[df_final['tasa anual (%)'] == "Tarjetas de crédito cerradas5", 'tasa anual (%)'] = 'Consumo - Tarjeta de crédito'
df_final.loc[df_final['tasa anual (%)'] == "Tarjetas de crédito abiertas4", 'tasa anual (%)'] = 'Consumo - Tarjeta de crédito'
df_final.loc[df_final['tasa anual (%)'] == "Tarjetas de Crdito", 'tasa anual (%)'] = 'Consumo - Tarjeta de crédito'

# Ahora filtramos la base de datos para que solo cuente con los tipos de créditos de interés
condition_1 = df_final['tasa anual (%)'] == "Préstamos hipotecarios para vivienda"
condition_2 = df_final['tasa anual (%)'] == "Consumo - Tarjeta de crédito"
df_filtrada = df_final[condition_1 | condition_2 ]


C:\Users\canun\AppData\Local\Temp\ipykernel_6484\340839837.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["year"].iloc[cont] = obs.strftime("%Y")


In [7]:
# Como la base de datos la tenemos en formato wide, debemos hacer un reshape wide to long utilizando las variables con los nombres de cada banco
melt_df_filtrada = df_filtrada.melt(id_vars=['year', 'tasa anual (%)'], value_vars=[ 'bbva', 'comercio', 'crédito', 'pichincha', 'bif', 'scotiabank', 'citibank', 'interbank', 'mibanco', 'gnb', 'falabella', 'santander', 'ripley', 'alfin', 'icbc', 'bank of china', 'trabajo', 'banco de comercio', 'banco de credito', 'banco pichincha', 'banbif', 'b santander central', 'standard chartered', 'banco sudamericano', 'wiese sudameris', 'bank boston', 'bnp paribas el', 'citibank del peru', 'scotiabank peru', 'hsbc(*)', 'hsbc', 'banco falabella', 'santander peru', 'banco ripley', 'alfin banco', 'deutsche bank peru', 'continental', 'financiero', 'azteca', 'deutsche', 'azteca  *', 'cencosud', 'banco gnb', 'financiero  *', 'cat', 'bci'], value_name='tasa')

# Como a lo largo de los años, los bancos han cambiado de nombre o se han escrito de manera distinta, generamos un diccionario que contiene los nombres de los bancos para luego poder realizar reemplazos a partir de este diccionario.
dict_bancos = {'continental': 'bbva', 'banco de comercio': 'comercio', 'banco de credito': 'crédito', 'banco pichincha': 'pichincha', 'financiero': 'pichincha', 'financiero  *': 'pichincha', 'banbif': 'bif', 'scotiabank peru': 'scotiabank', 'banco sudamericano': 'scotiabank', 'wiese sudameris': 'scotiabank', 'citibank del peru': 'citibank', 'banco gnb': 'gnb', 'hsbc(*)': 'gnb', 'hsbc': 'gnb', 'banco falabella': 'falabella', 'b santander central': 'santander', 'santander peru': 'santander', 'banco ripley': 'ripley', 'alfin banco': 'alfin', 'azteca': 'alfin', 'azteca  *': 'alfin', 'deutsche bank peru': 'deutsche', 'trabajo': 'trabajo', 'standard chartered': 'standard chartered', 'bank boston': 'bank boston', 'bnp paribas el': 'bnp paribas el', 'cencosud': 'cencosud', 'cat': 'cat', 'bci': 'bci', 'bbva':'bbva', 'comercio':'comercio', 'crédito':'crédito', 'pichincha':'pichincha', 'bif':'bif', 'scotiabank':'scotiabank', 'citibank':'citibank', 'gnb':'gnb' , 'falabella':'falabella', 'santander':'santander' , 'ripley':'ripley', 'alfin':'alfin', 'deutsche' :'deutsche', 'interbank' : 'interbank', 'icbc' : 'icbc', 'bank of china' : 'bank of china', 'mibanco':'mibanco'}

# Reemplazamos los valores a partir del diccionario creado
melt_df_filtrada['variable'] = melt_df_filtrada['variable'].map(dict_bancos)


In [11]:
# Hemos identificado que en algunos años no hay datos ('NaN' o '-' o 's.i') por lo tanto, realizamos un cambio de las variables a valores numéricos utilizanod la función pd.to_numeric
melt_df_filtrada['tasa'] = pd.to_numeric(melt_df_filtrada['tasa'], errors='coerce')

# Ahora, como debemos generar una base de datos donde únicamente se muestren las 5 mayores tasas de cada año, hemos generado un bucle a partir de dos listas. La primera "year_list" contiene todos los valores únicos de la variable "year" de la base de datos. La segunda "tasa_list" contiene los valores de los tipos de crédito que estamos analizando.

year_list = list(set(melt_df_filtrada['year']))
tasa_list = list(set(melt_df_filtrada['tasa anual (%)']))

# Generamos un contador que nos va a servir únicamente para poder generar la base de datos final 'df_top5'. Es decir, para que solo realice la acción de crear df_top5 en el primer paso del bucle y para lso demás concatene los df generados en el bucle.
contador3 = 0
for year in year_list:
    for rate in tasa_list:
        contador3+=1                                                    # Hacemos que crezca el contador
        condition_3 = melt_df_filtrada['year'] == year                  # Generamos una condición para filtrar para el año en mención
        condition_4 = melt_df_filtrada['tasa anual (%)'] == rate        # Generamos una condición para filtrar para el tipo de crédito
        df_alterna = melt_df_filtrada[condition_3 & condition_4]        # Filtramos la base de datos al año y tipo de crédito
        if rate == "Consumo - Tarjeta de crédito":                      # Generamos un condicional para que si el bucle en el que estamos es de crédito de consumo, ordene la base de datos en forma ascendente y al contrario si estamos en el bucle de crédito hipotecario
            df_alterna = df_alterna.sort_values(by=['tasa anual (%)' , 'tasa'], ascending=[True, False])
        else:
            df_alterna = df_alterna.sort_values(by=['tasa anual (%)' , 'tasa'], ascending=[False, False])
        df_alterna = df_alterna.drop_duplicates()
        df_alterna = df_alterna.iloc[0:5, :]                            #Filtramos para quedarnos solo con los 5 primeros valores
        if contador3 == 1 :                                             #Utilizamos el contador generado al inicio del bucle para crear o concatenar las bases de datos
            df_top5 = df_alterna
        else:
            df_top5 = pd.concat([df_top5 , df_alterna])

# Finalmente, con la base de datos final realizamos el cambio de nombres y ordenamos la data en función al año, tipo de crédito y valor de la tasa
df_top5 = df_top5.rename(columns={ 'tasa anual (%)':'rate_concept', 'variable':'banks', 'tasa':'rate_value'})
df_top5 = df_top5.sort_values(by=['year', 'rate_concept' , 'rate_value'], ascending=[True, False, False])

df_top5

,year,rate_concept,banks,rate_value
9155,2001,Préstamos hipotecarios para vivienda,mibanco,14.25
1147,2001,Préstamos hipotecarios para vivienda,comercio,8.24
3435,2001,Préstamos hipotecarios para vivienda,pichincha,8.20
10299,2001,Préstamos hipotecarios para vivienda,gnb,7.69
4579,2001,Préstamos hipotecarios para vivienda,bif,6.88
...,...,...,...,...
12576,2022,Consumo - Tarjeta de crédito,falabella,79.81
12571,2022,Consumo - Tarjeta de crédito,falabella,79.16
12566,2022,Consumo - Tarjeta de crédito,falabella,79.03
12581,2022,Consumo - Tarjeta de crédito,falabella,78.89


# Answer 4:

In [31]:
# Utilizando el paquete os y la función mkdir generamos la carpeta nueva en el repositorio de GitHub del curso.
os.mkdir(r"../../_output\sbs\group_9")
# Nombramos el path para utilizarlo en el bucle final
relative_path = Path(r"../../_output\sbs\group_9")

# Debido a que existe la posibilidad de que los nombres de los bancos tengan espacio, generamos un buclea para eliminar los espacios en blanco y reemplazarlos por "_"
bank_list = list(set(df_top5['banks']))
contador4 =0
for bank in bank_list:
    bank_list[contador4] = bank_list[contador4].replace(" ", "_")
    contador4+=1

# Finalmente, generamos un bucle para generar un archivo csv para cada banco que se encuentre en la base de los top5 de cada año y tipo de crédito. Esto lo hacemos con la función to.csv y utilizando el relative path generado anteriormente
for bank in bank_list:
    condition = df_top5['banks'] == bank
    bank_alterna = df_top5[condition]
    bank_alterna.to_csv(f"{relative_path}/{bank}.csv", index= False )
